In [1]:
# !pip install pyradiomics

In [2]:
import os  # needed navigate the system to get the input data
from radiomics import featureextractor  # This module is used for interaction with pyradiomics
import radiomics
import numpy as np
import SimpleITK as sitk
import cv2, pkgutil
import pandas as pd
import xlsxwriter
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
import matplotlib.pyplot as plt
from csv import writer

In [3]:

# # print(os.listdir(os.path.dirname(radiomics.__file__)))

# for _, mod, _ in pkgutil.iter_modules([os.path.dirname(radiomics.__file__)]):
#     print(mod)
    

_cmatrices
_cshape
_version
base
featureextractor
features
firstorder
generalinfo
glcm
gldm
glrlm
glszm
imageoperations
ngtdm
scripts
shape
shape2D


In [4]:
def FindBoundingBox( contour ):
    x,y,w,h = cv2.boundingRect( contour )
    return x,y,w,h



def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)
        

        
def color_define_contour(label):
    
    if label==0:
        color = (255, 0, 0)
    
    elif label==1:
        color = (255, 255, 0)
    
    elif label==2:
        color = (0, 0, 255)
    
    elif label==3:
        color = (0, 100, 0)        
        
    return color
        

        


In [5]:
def load_pickle(data_file,q):
    # Reading dictionary to load train and test data
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']

    print('\nPickle Loaded Successfully!')

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)

    return X_train,Y_train



def save_pickle(x_train,y_train, data_file):
    # Reading dictionary to save train and test data
    print('\nTrying to save pickle to %s' % data_file)

    X_train = x_train
    Y_train = y_train
    del x_train,y_train

    # creating dictionary to store train and test data
    datasets = {
        'dataset': {'X_train': X_train,'Y_train': Y_train}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.

    print('\nPickle Saved Successfully!')

    
    

In [6]:
##################### Our Technqiue #####################
patch_size = (128, 128)
step = 200

print('\nSelect Organ')
option_Organ = int(input("\n 1-Breast \n 2-Kidney \n 3-Lung \n 4-Prostate \n"))
while option_Organ  not in (1,2,3,4,5):
    option_Organ = int(input("\n 1-Breast \n 2-Kidney \n 3-Lung \n 4-Prostate \n"))

if option_Organ==1:
    Organ = "Breast"
elif option_Organ==2:
    Organ = "Kidney"
elif option_Organ==3:
    Organ = "Lung"
elif option_Organ==4:
    Organ = "Prostate"
    
    

data_folder_R = "D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/"

Mode = "Train"
path_ = "D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_" + str(patch_size[0]) + "_" + str(step) + "/Data_Patches/" + str(Organ)

# # Mode = "Test"
# path_ = "D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Testing_Data_Classwise_" + str(patch_size[0]) + "_" + str(step) + "/Data_Patches/" + str(Organ) 

data_folder1 = "/Images/"
path1  = path_ + data_folder1
folder1 = sorted(os.listdir(path1))
print('\nTotal Images: ', len(folder1))

data_folder2 = "/Masks/"
path2  = path_ + data_folder2
folder2 = sorted(os.listdir(path2))
print('Total Masks: ', len(folder2))

folder_diff = [file for file in folder1 if file.endswith(".png")]
folder_label = [file for file in folder2 if file.endswith(".png")]

print(len(folder_diff))
print(len(folder_label))



Select Organ

 1-Breast 
 2-Kidney 
 3-Lung 
 4-Prostate 
4

Total Images:  46
Total Masks:  46
46
46


In [7]:
print(path1)
print(path2)

D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_128_200/Data_Patches/Prostate/Images/
D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_128_200/Data_Patches/Prostate/Masks/


In [8]:
from natsort import natsorted

No_Count = []

for seq in np.asarray(natsorted(folder_diff)): 
    Images_No = seq.split('_')[0]
    No_Count.append(Images_No)

temp = np.unique(No_Count)
print(temp)

print("\nLength: ",len(temp))

print("\n")
for seq in natsorted(temp): 
    print(seq)

['Image-0' 'Image-1' 'Image-2' 'Image-3' 'Image-4' 'Image-5' 'Image-6'
 'Image-7']

Length:  8


Image-0
Image-1
Image-2
Image-3
Image-4
Image-5
Image-6
Image-7


In [9]:
Count_L = 0

Images = []
Labels = []

for seq in np.asarray(natsorted(folder_diff)): 

    imagePath = os.path.join(path1, str(natsorted(folder_diff)[Count_L]))    
    img=cv2.imread(os.path.join(imagePath))
    Images.append(img)
    
    for temp_ in natsorted(temp): 

        if temp_ == seq.split('_')[0]:
            
            Image_No = int(seq.split('_')[0].split('-')[1])
            Mask_No = int(seq.split('_')[1].split('-')[1])
            C = seq.split('_')[2]

            if C=="Type-E":
                Labels_No = 0
                Labels.append(Labels_No)

            elif C=="Type-L":
                Labels_No = 1
                Labels.append(Labels_No)

            elif C=="Type-M":
                Labels_No = 2
                Labels.append(Labels_No)

            elif C=="Type-N":
                Labels_No = 3
                Labels.append(Labels_No)

    Count_L = Count_L + 1


In [11]:
from six.moves import cPickle as pickle

Images_ = np.asarray(Images) 
print(Images_.shape)

Labels_ = np.asarray(Labels) 
print(Labels_.shape)
print(Labels_)


(46, 128, 128, 3)
(46,)
[0 1 3 0 0 0 1 1 1 2 2 2 0 0 0 1 1 1 2 2 2 0 1 3 0 0 0 0 1 2 0 1 1 3 1 1 2
 2 2 3 1 1 2 2 2 3]


In [13]:
data_file = 'January/PICKLESNEW/Classify_B_' + str(Mode) + '_' + str(Organ) + '.pickle'   
print(data_file)    

January/PICKLESNEW/Classify_Train_Texture_New_Prostate.pickle


In [14]:
if Mode == "Train":
    resultDir = os.path.join(data_folder_R, "Augment_January_23", "Result_Train/")
    resultDir_Labels = os.path.join(data_folder_R, "Augment_January_23", "Result_Train_Labels/")

elif Mode == "Test":
    resultDir = os.path.join(data_folder_R, "Augment_January_23", "Result_Test/")
    resultDir_Labels = os.path.join(data_folder_R, "Augment_January_23", "Result_Test_Labels/")

    
if not os.path.exists(resultDir):
    os.mkdir(resultDir)

resultDir_O = os.path.join(resultDir,  str(Organ))

if not os.path.exists(resultDir_O):
    os.mkdir(resultDir_O)

if not os.path.exists(resultDir_Labels):
    os.mkdir(resultDir_Labels)

resultExcelPath_Labels = os.path.join(resultDir_Labels,  str(Organ) + ".csv")
print(resultExcelPath_Labels)

i_exception = []
contours_ = []
Save_No = 0
    
print("\n\t\t\t\t===== Processing Image =====")
for i in range(len(folder_diff)):
    
    if Mode == "Train":
        print("\nImage #: ", str(Save_No))
        
        imagePath = os.path.join(path1, str(folder_diff[i]))
        labelPath = os.path.join(path2, str(folder_label[i]))

        gray_img = cv2.imread(imagePath, cv2.IMREAD_ANYDEPTH)  # original_img is the numpy.ndarray class
        label_img = cv2.imread(labelPath, -1)  # If there is no -1, read as 0 as 8 bits, and read as the original image with -1

        columns_list = ['Feature']

        try:
            resultExcelPath = os.path.join(resultDir_O, str(i) + ".csv")
            Save_No = Save_No + 1
            
            contours, hierarchy = cv2.findContours(label_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE) # Get the number and area of labels.
            blank_image = np.zeros((label_img.shape[0], label_img.shape[1],3), np.uint8) # shape[1] is the width

            for k in range(len(contours)):
                copy_label_image = np.zeros((label_img.shape[0], label_img.shape[1]),np.uint8)  # To extract only that label

                color = color_define_contour(Labels_[i])

                cv2.polylines(blank_image, contours[k], True, color, 2)
                cv2.fillPoly(copy_label_image, [contours[k]], (255)) # To extract only the corresponding mask

                box = FindBoundingBox(contours[k] ) # Box extraction for gray image crop
                cv2.rectangle( blank_image, (box[0], box[1]), ( box[0]+box[2], box[1]+box[3]), (255,0,255), 2 )

                image_area_roi = gray_img[box[1]:box[1]+box[3], box[0]:box[0]+box[2]] # gray image crop
                label_img_roi = copy_label_image[box[1]:box[1]+box[3], box[0]:box[0]+box[2]] # label image crop

                # ndarray class is changed to 3D sitk format, RadioMics only receives 3D simpleITK object as input
                image_area_roi_arr = np.expand_dims(image_area_roi, axis=0)
                original_img_im3d = sitk.GetImageFromArray(image_area_roi_arr)
                label_img_roi_arr = np.expand_dims(label_img_roi, axis=0)
                label_img_im3d = sitk.GetImageFromArray(label_img_roi_arr)

                extractor = featureextractor.RadiomicsFeatureExtractor()     # For 2D Shapes force2D=True
                radiomics.setVerbosity(60)

                try:
                    results = extractor.execute(original_img_im3d, label_img_im3d, label=255)
                except NameError:
                    print("Mask is not defined")
                except:
                    pass

                # Show output
                featureDictionary={} # Initialize the dictionary to extract the result
                features =[]
                value_title = str(k)
                values =[]
                for featureName in results.keys():
                    if featureName.find('diagnostics_') == -1 :
                        new_feature_str = featureName[9:]
                        features.append(new_feature_str )
                        values.append(float(results[featureName]))

                if k == 0 :
                    computed_data = {'Feature': features, '0': values}
                    df = pd.DataFrame(data=computed_data, columns=['Feature', '0'])
                else:
                    series_add = pd.Series( values, name = value_title )
                    df = pd.concat([df, series_add], axis = 1)

            print("# of Contours: ", len(contours))
            print("# of Hierarchy: ", len(hierarchy))
            contours_.append(df.shape[1]-1)
            print(contours_)
            df.to_csv(resultExcelPath, index = False, sep=',', line_terminator = '\r')
            
        except:
            i_exception.append(i)
            pass


    
print("\n")
print("Exceptions: ",i_exception)
print("Length: ",len(i_exception), "\n")

append_list_as_row(resultExcelPath_Labels, contours_)  

Images_ = np.delete(Images_, i_exception, axis=0)
Labels_ = np.delete(Labels_, i_exception)

print(Images_.shape)
print(Labels_.shape)
save_pickle(Images_, Labels_, data_file)




D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv

				===== Processing Image =====

Image #:  0

Image #:  1
# of Contours:  1
# of Hierarchy:  1
[1]

Image #:  2
# of Contours:  34
# of Hierarchy:  1
[1, 34]

Image #:  3
# of Contours:  49
# of Hierarchy:  1
[1, 34, 49]

Image #:  4
# of Contours:  93
# of Hierarchy:  1
[1, 34, 49, 93]

Image #:  5
# of Contours:  64
# of Hierarchy:  1
[1, 34, 49, 93, 64]

Image #:  6
# of Contours:  67
# of Hierarchy:  1
[1, 34, 49, 93, 64, 67]

Image #:  7
# of Contours:  54
# of Hierarchy:  1
[1, 34, 49, 93, 64, 67, 54]

Image #:  8
# of Contours:  8
# of Hierarchy:  1
[1, 34, 49, 93, 64, 67, 54, 8]

Image #:  9
# of Contours:  1
# of Hierarchy:  1
[1, 34, 49, 93, 64, 67, 54, 8, 1]

Image #:  10
# of Contours:  21
# of Hierarchy:  1
[1, 34, 49, 93, 64, 67, 54, 8, 1, 21]

Image #:  11
# of Contours:  1
# of Hierarchy:  1
[1, 34, 49, 93, 64, 67, 54, 8, 1, 21, 1]

Image #:  12
# of Contours:  49
# of Hiera